In [5]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from imutils import paths
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import numpy as np
import seaborn as sns
from sklearn.metrics import classification_report
import pandas as pd
import time
from os.path import isfile, join

In [6]:
# RUN_ENV='KATE'
# RUN_ENV='JN'
# RUN_ENV='MK'
RUN_ENV='GERRY'

In [7]:
APP_FOLDER=''
if RUN_ENV=='JN':
    APP_FOLDER='/Users/jialingcai/Documents/607_608_project/'

if RUN_ENV=='GERRY':
    APP_FOLDER='C:/Final/'

METADATA_FOLDER=APP_FOLDER+'Metadata/'
NEW_DATA_FOLDER=APP_FOLDER+'NewData/'
DATASET1_PATH=APP_FOLDER+'/SourceData/DataSet1/'
##for dataset 1
XMLSCHEMA_FILE=METADATA_FOLDER+'dataset1.xsd'

DATASET2_PATH=APP_FOLDER+'/SourceData/DataSet2/'
##

In [8]:
path_with_mask =NEW_DATA_FOLDER+'face_with_mask'
path_without_mask = NEW_DATA_FOLDER+'face_no_mask'
files_with_mask = os.listdir(path_with_mask)
files_without_mask = os.listdir(path_without_mask)
print(files_with_mask[1])

1803_0_face_with_mask.jpg


In [9]:
def get_image_info(img_path):
    image_obj = load_img(img_path)
    newsize=(224,224)
    new_img_info = image_obj.resize(newsize)

    return (new_img_info)

X1 = []
# X_without_mask=[]
# y_label_without=[]

for i in range(len(files_without_mask)):
    j = 1
    img_path = join(path_without_mask,files_without_mask[i])#path_with_mask+'/'+files_with_mask[i]
    x = get_image_info(img_path)
    image = img_to_array(x)
    X1.append({'name': j , 'value': image })

In [10]:
X1 = np.array(X1)
X1[1].get('name')
X1[1].get('value').shape

(224, 224, 3)

In [11]:
%%time
X0=[]
for i in range(len(files_with_mask)):
    j=0
    img_path = join(path_with_mask,files_with_mask[i])#path_with_mask+'/'+files_with_mask[i]
    x = get_image_info(img_path)
    image = img_to_array(x)
    X0.append({'name': j , 'value': image })

import random
from sklearn.neighbors import KNeighborsClassifier

data = np.concatenate((X1,X0),axis=0)
data.shape
random.shuffle(data)
data.shape
#WE LABEL WITHMASK AS 0, WITHOUT MASK AS 1
X = [data[i].get('value') for i in range(5283)]
np.array(X).shape
#print(data[1])

X_tr, X_te, y_tr, y_te = train_test_split(np.array([data[i].get('value') for i in range(5283)]), 
                                          np.array([data[i].get('name') for i in range(5283)]),
                                         test_size=0.2,
                                         random_state=42)
X_tr.shape,y_tr.shape
X_tr = X_tr.reshape(X_tr.shape[0],X_tr.shape[1]*X_tr.shape[2]*X_tr.shape[3])
X_te = X_te.reshape(X_te.shape[0],X_te.shape[1]*X_te.shape[2]*X_te.shape[3])

model = KNeighborsClassifier(n_neighbors=5)
model.fit(X_tr,y_tr)
ypre = model.predict(X_te)
target_names = ['with_mask','without_mask']

clf_report = classification_report(y_te,
                                   ypre,
                                   target_names=target_names,
                                   output_dict=True)



Wall time: 53.8 s


In [12]:
from joblib import dump
dump(model,filename='KNN_Best.joblib')

['KNN_Best.joblib']